What is pytorch?

Python-based scientific computing package for two audiences
- Replacement for numpy to use power of GPUs
- Deep learning research platform that provides max flexibility and speed

Tensors

Tensors are similar to numpy's ndarrays, with the addition being that Tensors can also be used on a GPU to accelerate computing

In [1]:
from __future__ import print_function
import torch

ImportError: No module named 'torch'

In [ ]:
# construct a 5x3 matrix, uninitialized
x = torch.Tensor(5, 3)
print(x)

# construct a randomly initialized matrix
x = torch.rand(5, 3)
print(x)

# get its size
print(x.size())

# operations -- multiple syntaxes for operations

# addition 1
y = torch.rand(5, 3)
print(x + y)

# addition 2
print(torch.add(x, y))

# addition: giving an output tensor
result = torch.Tensor(5, 3)
torch.add(x, y, out=result)
print(result)

# addition: in-place
# adds x to y
y.add(x)
print(y)

# can use standard numpy-like indexing with all bells and whistles
print(x[:, 1])


Numpy Bridge

Converting a torch Tensor to a numpy array and vice versa is a breeze

The torch Tensor and numpy array will share their underlying memory locations, and changing one will change the other

In [ ]:
# Converting torch Tensor to numpy Array
a = torch.ones(5)
print(a)

b = a.numpy()
print(b)

# see how the numpy array changed in value
a.add_(1)
print(a)
print(b)

# Converting numpy Array to torch Tensor
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

# All the Tensors on the CPU except a CharTensor support converting to NumPy and back

# CUDA Tensors
# Tensors can be moved onto GPU using the `.cuda` function

# let us run this cell if CUDA is available
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x + y
    

Autograd

The autograd package provides automatic differentiation for all operations on Tensors.  It's a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Variable

`autograd.Variable` is the central class of the package.  It wraps a Tensor, and supports nearly all of operations defined on it.  Once you finish your computation you can call `.backward()` and have all the gradients computed automatically.

You can access the raw tensor through the `.data` attribute, while the gradient wrt this variable is accumulated into `.grad`

There's one more function important to autograd implementation: `Function`

`Variable` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation.  Each variable has a `.creator` attribute that references a `Function` that has created the `Variable` (except for Variables created by the user -- their `creator is None`)

If you want to compute the derivatives, you can call `.backward()` on a `Variable`.  If `Variable` is a scalar (ie, it holds a one element data), you don't need to specify any arguments to `backward()`, however if it has more elements, you need to specify a `grad_output` argument that a tensor of matching shape

In [ ]:
import torch
from torch.autograd import Variable

# create a variable
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

# do an operation of variable
y = x + 2
print(y)

# y was created as a result of an operation, so it has a creator
print(y.creator)

# do more operations on y
z = y * y * 3
out = z.mean()
print(z, out)


In [ ]:
# Gradients

# let's backprop now `out.backward()` is equivalent of `out.backward(torch.Tensor([1.0]))

out.backward()

# print gradients d(out)/dx

print(x.grad)

# You should have gotten a matrix of 4.5


Neural Networks

Neural networks can be constructed using the `torch.nn` package.

Now that you had a glimpse of `autograd`, `nn` depends on `autograd` to define models and differentiate them. An `nn.Module` contains layers, and a method `forward(input)` that returns the `output`

A simple feed-forward network takes input, feeds it thru several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:
- Define the nn that has some learnable parameters (weights)
- Iterate over a dataset of inputs
- Process input thru the network
- Compute the loss (how far the output is from being correct)
- Propagate gradients back into the network's parameters
- Update the weights of the network, typically using a simple update rule:
`weight = weight - learning_rate * gradient`


In [ ]:
# define the network

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)


You just have to define the `forward` function, and the `backward` function (where gradients are computed) is automatically defined for you using `autograd`.  You can use any of the Tensor operations in the `forward` function

The learnable parameters of a model are returned by `net.parameters()`


In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

# the input to the forward is an autograd.Variable, and so is the output

input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

# zero the gradient buffers of all parameters and backprops with random grandients

net.zero_grad()
out.backward(torch.randn(1, 10))

`torch.nn` only supports mini-batches.  The entire `torch.nn` package only supports inputs that are a mini-batch of samples, and not a single sample.

For example, `nn.Conv2d` will take a 4D Tensor of `nSamples x nChannels x Height x Width`.

If you have a single sample, just use `input.unsqueeze(0)` to add a fake batch dimension.

Recap:



Loss function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the nn package.  A simple loss is `nn.MSELoss` which computes the mean-squared error between the input and the target.


In [ ]:
# example
output = net(input)
target = Variable(torch.arange(1, 11)) # dummy target, for example
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Backprop

To backprop the error all we have to do is to `loss.backward()`. You need to clear the existing gradients thru, else gradients will be accumulated to existing gradients

Now we shall call `loss.backward()` and have a look at conv1's bias gradients before and after the backward.

In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

Update the weights

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

`weight = weight - learning_rate * gradient`

We can implement this using simple python code:

In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: torch.optim that implements all these methods. Using it is very simple:

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

Training a classifier

Data

Generally image, text, audio, or video data, you can use standard python packages that load data into a numpy array.  Then you can convert this array into a `torch.*Tensor`

- For images, packages such as Pillow, OpenCV are useful.
- For audio, packages such as scipy and librosa
- For text, either raw Python or Cython based loading, or NLTK and SpaCy are useful.

Specifically for vision, we have created a package called torchvision, that has data loaders for common datasets such as Imagenet, CIFAR10, MNIST, etc. and data transformers for images, viz., torchvision.datasets and torch.utils.data.DataLoader.

This provides a huge convenience and avoids writing boilerplate code.

For this tutorial, we will use the CIFAR10 dataset. It has the classes: ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’. The images in CIFAR-10 are of size 3x32x32, i.e. 3-channel color images of 32x32 pixels in size.

